<a href="https://colab.research.google.com/github/Yyyyyjkim/review_analysis/blob/master/Konlpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install konlpy

In [0]:
import pandas as pd
import pickle
import konlpy
import re

In [0]:
with open('/content/drive/My Drive/comment_analysis/data/review.pickle', 'rb') as f:
    review = pickle.load(f)

In [0]:
review.columns = ['ranking','date','desc','asc']

In [0]:
review_ranking = [re.sub('[\[\]!"#$%&\'()*+,/:;<=>?@\^_`{|}~-✔️🙏🏻👍🤗💕😘]|BEST\n|재구매\n','',i) for i in review['ranking']]
review_ranking = [re.sub('\n',' ',i) for i in review_ranking]

In [0]:
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.tag import Komoran
from konlpy.tag import Okt

In [8]:
hannanum = Hannanum() 
hannanum.analyze  #구(Phrase) 분석
hannanum.morphs   #형태소 분석
hannanum.nouns    #명사 분석
hannanum.pos      #형태소 분석 태깅

<bound method Hannanum.pos of <konlpy.tag._hannanum.Hannanum object at 0x7f4a87bc0e48>>

In [12]:
hannanum.morphs(review.iloc[0,0])

'BEST'